In [1]:
from rosetta.protocols.rosetta_scripts import *
from pyrosetta import *
from pyrosetta import PyMOLMover
from rosetta.core.pack.task import *
init()

<ipython-input-1-3d9f072db938>:1: UserWarning: Import of 'rosetta' as a top-level module is deprecated and may be removed in 2018, import via 'pyrosetta.rosetta'.
  from rosetta.protocols.rosetta_scripts import *


PyRosetta-4 2020 [Rosetta PyRosetta4.Release.python38.linux 2020.50+release.1295438cd4bd2be39c9dbbfab8db669ab62415ab 2020-12-12T00:30:01] retrieved from: http://www.pyrosetta.org
(C) Copyright Rosetta Commons Member Institutions. Created in JHU by Sergey Lyskov and PyRosetta Team.
core.init: Checking for fconfig files in pwd and ./rosetta/flags
core.init: Rosetta version: PyRosetta4.Release.python38.linux r274 2020.50+release.1295438 1295438cd4bd2be39c9dbbfab8db669ab62415ab http://www.pyrosetta.org 2020-12-12T00:30:01
core.init: command: PyRosetta -ex1 -ex2aro -database /home/csaylan/anaconda3/lib/python3.8/site-packages/pyrosetta-2020.50+release.1295438-py3.8-linux-x86_64.egg/pyrosetta/database
basic.random.init_random_generator: 'RNG device' seed mode, using '/dev/urandom', seed=-1217673224 seed_offset=0 real_seed=-1217673224
basic.random.init_random_generator: RandomGenerator:init: Normal mode, seed=-1217673224 RG_type=mt19937


In [2]:
pose = pose_from_pdb("../../Results/Simulations/FKBP_RYR2/5go9_model/preparation/merged_ryr2.pdb")

core.chemical.GlobalResidueTypeSet: Finished initializing fa_standard residue type set.  Created 985 residue types
core.chemical.GlobalResidueTypeSet: Total time to initialize 1.08246 seconds.
core.import_pose.import_pose: File '../../Results/Simulations/FKBP_RYR2/5go9_model/preparation/merged_ryr2.pdb' automatically determined to be of type PDB
core.io.pose_from_sfr.PoseFromSFRBuilder: [ WARNING ] discarding 1 atoms at position 1 in file ../../Results/Simulations/FKBP_RYR2/5go9_model/preparation/merged_ryr2.pdb. Best match rsd_type:  HIS:NtermProteinFull
core.io.pose_from_sfr.PoseFromSFRBuilder: [ WARNING ] discarding 1 atoms at position 311 in file ../../Results/Simulations/FKBP_RYR2/5go9_model/preparation/merged_ryr2.pdb. Best match rsd_type:  ASP
core.io.pose_from_sfr.PoseFromSFRBuilder: [ WARNING ] discarding 2 atoms at position 430 in file ../../Results/Simulations/FKBP_RYR2/5go9_model/preparation/merged_ryr2.pdb. Best match rsd_type:  GLN
core.io.pose_from_sfr.PoseFromSFRBuilder

core.io.pose_from_sfr.PoseFromSFRBuilder: [ WARNING ] discarding 2 atoms at position 706 in file ../../Results/Simulations/FKBP_RYR2/5go9_model/preparation/merged_ryr2.pdb. Best match rsd_type:  PHE
core.io.pose_from_sfr.PoseFromSFRBuilder: [ WARNING ] discarding 3 atoms at position 707 in file ../../Results/Simulations/FKBP_RYR2/5go9_model/preparation/merged_ryr2.pdb. Best match rsd_type:  GLU
core.io.pose_from_sfr.PoseFromSFRBuilder: [ WARNING ] discarding 1 atoms at position 708 in file ../../Results/Simulations/FKBP_RYR2/5go9_model/preparation/merged_ryr2.pdb. Best match rsd_type:  VAL
core.io.pose_from_sfr.PoseFromSFRBuilder: [ WARNING ] discarding 2 atoms at position 709 in file ../../Results/Simulations/FKBP_RYR2/5go9_model/preparation/merged_ryr2.pdb. Best match rsd_type:  LEU
core.io.pose_from_sfr.PoseFromSFRBuilder: [ WARNING ] discarding 3 atoms at position 710 in file ../../Results/Simulations/FKBP_RYR2/5go9_model/preparation/merged_ryr2.pdb. Best match rsd_type:  MET
core.

core.io.pose_from_sfr.PoseFromSFRBuilder: [ WARNING ] discarding 3 atoms at position 745 in file ../../Results/Simulations/FKBP_RYR2/5go9_model/preparation/merged_ryr2.pdb. Best match rsd_type:  SER
core.io.pose_from_sfr.PoseFromSFRBuilder: [ WARNING ] discarding 4 atoms at position 746 in file ../../Results/Simulations/FKBP_RYR2/5go9_model/preparation/merged_ryr2.pdb. Best match rsd_type:  ARG
core.io.pose_from_sfr.PoseFromSFRBuilder: [ WARNING ] discarding 2 atoms at position 747 in file ../../Results/Simulations/FKBP_RYR2/5go9_model/preparation/merged_ryr2.pdb. Best match rsd_type:  LEU
core.io.pose_from_sfr.PoseFromSFRBuilder: [ WARNING ] discarding 5 atoms at position 748 in file ../../Results/Simulations/FKBP_RYR2/5go9_model/preparation/merged_ryr2.pdb. Best match rsd_type:  LYS
core.io.pose_from_sfr.PoseFromSFRBuilder: [ WARNING ] discarding 3 atoms at position 749 in file ../../Results/Simulations/FKBP_RYR2/5go9_model/preparation/merged_ryr2.pdb. Best match rsd_type:  GLN
core.

core.io.pose_from_sfr.PoseFromSFRBuilder: [ WARNING ] discarding 3 atoms at position 784 in file ../../Results/Simulations/FKBP_RYR2/5go9_model/preparation/merged_ryr2.pdb. Best match rsd_type:  MET
core.io.pose_from_sfr.PoseFromSFRBuilder: [ WARNING ] discarding 3 atoms at position 785 in file ../../Results/Simulations/FKBP_RYR2/5go9_model/preparation/merged_ryr2.pdb. Best match rsd_type:  GLN
core.io.pose_from_sfr.PoseFromSFRBuilder: [ WARNING ] discarding 1 atoms at position 786 in file ../../Results/Simulations/FKBP_RYR2/5go9_model/preparation/merged_ryr2.pdb. Best match rsd_type:  THR
core.io.pose_from_sfr.PoseFromSFRBuilder: [ WARNING ] discarding 3 atoms at position 787 in file ../../Results/Simulations/FKBP_RYR2/5go9_model/preparation/merged_ryr2.pdb. Best match rsd_type:  SER
core.io.pose_from_sfr.PoseFromSFRBuilder: [ WARNING ] discarding 1 atoms at position 788 in file ../../Results/Simulations/FKBP_RYR2/5go9_model/preparation/merged_ryr2.pdb. Best match rsd_type:  THR
core.

In [3]:
working_pose = pose.clone()

In [4]:
scorefxn = create_score_function("ref2015_cart")

core.energy_methods.CartesianBondedEnergy: Initializing IdealParametersDatabase with default Ks=300 , 80 , 80 , 10 , 80
basic.io.database: Database file opened: scoring/score_functions/bondlength_bondangle/default-lengths.txt
core.energy_methods.CartesianBondedEnergy: Read 759 bb-independent lengths.
basic.io.database: Database file opened: scoring/score_functions/bondlength_bondangle/default-angles.txt
core.energy_methods.CartesianBondedEnergy: Read 1434 bb-independent angles.
basic.io.database: Database file opened: scoring/score_functions/bondlength_bondangle/default-torsions.txt
core.energy_methods.CartesianBondedEnergy: Read 1 bb-independent torsions.
basic.io.database: Database file opened: scoring/score_functions/bondlength_bondangle/default-improper.txt
core.energy_methods.CartesianBondedEnergy: Read 529 bb-independent improper tors.


MoveMap

In [ ]:
# For a specific region of residues
res_action = pyrosetta.rosetta.core.select.residue_selector.NeighborhoodResidueSelector()
res_action.set_focus(str(673))
res_action.set_distance(20)
res_action.set_include_focus_in_subset(True) 

: 

In [6]:
# For a specific array of residues

#res_select = pyrosetta.rosetta.core.select.residue_selector.ResidueIndexSelector()
#res_select.set_index_range(1,10)

In [7]:
movemap = pyrosetta.rosetta.core.select.movemap.MoveMapFactory()
movemap.all_jumps(False)
movemap.all_bb(False)
movemap.add_bb_action(pyrosetta.rosetta.core.select.movemap.mm_enable, res_action)
movemap.add_chi_action(pyrosetta.rosetta.core.select.movemap.mm_enable, res_action)
movemap.set_cartesian(True) 

In [8]:
tf = TaskFactory()
tf.push_back(operation.InitializeFromCommandline())
tf.push_back(operation.RestrictToRepacking())

prevent_repacking_rlt = operation.PreventRepackingRLT()
prevent_subset_repacking = operation.OperateOnResidueSubset(prevent_repacking_rlt, res_action, True)
tf.push_back(prevent_subset_repacking)

packer = pyrosetta.rosetta.protocols.minimization_packing.PackRotamersMover() 
packer.score_function(scorefxn)
packer.task_factory(tf)

fastrelax = pyrosetta.rosetta.protocols.relax.FastRelax()
fastrelax.set_scorefxn(scorefxn)
fastrelax.cartesian(True)
fastrelax.set_movemap_factory(movemap)
fastrelax.set_task_factory(tf)

fastrelax.apply(working_pose) 

protocols.relax.RelaxScriptManager: Reading relax scripts list from database.
core.scoring.ScoreFunctionFactory: SCOREFUNCTION: ref2015
protocols.relax.RelaxScriptManager: Looking for MonomerRelax2019.txt
protocols.relax.RelaxScriptManager: ================== Reading script file: /home/csaylan/anaconda3/lib/python3.8/site-packages/pyrosetta-2020.50+release.1295438-py3.8-linux-x86_64.egg/pyrosetta/database/sampling/relax_scripts/MonomerRelax2019.txt ==================
protocols.relax.RelaxScriptManager: repeat %%nrepeats%%
protocols.relax.RelaxScriptManager: coord_cst_weight 1.0
protocols.relax.RelaxScriptManager: scale:fa_rep 0.040
protocols.relax.RelaxScriptManager: repack
protocols.relax.RelaxScriptManager: scale:fa_rep 0.051
protocols.relax.RelaxScriptManager: min 0.01
protocols.relax.RelaxScriptManager: coord_cst_weight 0.5
protocols.relax.RelaxScriptManager: scale:fa_rep 0.265
protocols.relax.RelaxScriptManager: repack
protocols.relax.RelaxScriptManager: scale:fa_rep 0.280
protoco

protocols.relax.FastRelax: CMD: scale:fa_rep  2582.31  0.381768  0.381768  0.55
core.pack.task: Packer task: initialize from command line()
core.pack.pack_rotamers: built 4710 rotamers at 160 positions.
core.pack.interaction_graph.interaction_graph_factory: Instantiating DensePDInteractionGraph
protocols.relax.FastRelax: CMD: repack  2582.47  0.381768  0.381768  0.55
core.optimization.LineMinimizer: [ ERROR ] Inaccurate G! step= 9.53674e-09 Deriv= -0.00147086 Finite Diff= 0.0160694
core.optimization.LineMinimizer: [ ERROR ] Inaccurate G! step= 1.1893e-06 Deriv= -41.1525 Finite Diff= 46.0249
core.optimization.Minimizer: Line search failed even after resetting Hessian; aborting at iter#1515
protocols.relax.FastRelax: CMD: min  2497.72  0.387811  0.387811  0.55
protocols.relax.FastRelax: MRP: 1  2497.72  2497.72  0.387811  0.387811
protocols.relax.FastRelax: CMD: accept_to_best  2497.72  0.387811  0.387811  0.55
protocols.relax.FastRelax: CMD: endrepeat  2497.72  0.387811  0.387811  0.55


protocols.relax.FastRelax: CMD: repack  560.297  0.378888  0.378888  0.022
protocols.relax.FastRelax: CMD: scale:fa_rep  586.757  0.378888  0.378888  0.02805
protocols.relax.FastRelax: CMD: min  -25.5893  0.401144  0.401144  0.02805
protocols.relax.FastRelax: CMD: coord_cst_weight  -25.5893  0.401144  0.401144  0.02805
protocols.relax.FastRelax: CMD: scale:fa_rep  1280.29  0.401144  0.401144  0.14575
core.pack.task: Packer task: initialize from command line()
core.pack.pack_rotamers: built 5400 rotamers at 162 positions.
core.pack.interaction_graph.interaction_graph_factory: Instantiating DensePDInteractionGraph
protocols.relax.FastRelax: CMD: repack  1275.3  0.401144  0.401144  0.14575
protocols.relax.FastRelax: CMD: scale:fa_rep  1366.12  0.401144  0.401144  0.154
protocols.relax.FastRelax: CMD: min  844.28  0.3845  0.3845  0.154
protocols.relax.FastRelax: CMD: coord_cst_weight  844.28  0.3845  0.3845  0.154
protocols.relax.FastRelax: CMD: scale:fa_rep  1655.24  0.3845  0.3845  0.307

In [9]:
working_pose.dump_pdb("../../Results/Simulations/FKBP_RYR2/5go9_model/preparation/relaxed_673.pdb")

True